In [1]:
import collections
import math
import torch
from torch import nn
from d2l import torch as d2l

In [4]:
class Encoder(nn.Module):
    """编码器-解码器架构的基本编码器接口"""
    def __init__(self,**kwargs):
        super(Encoder,self).__init__(**kwargs)
        
    def forward(self,X,*args):
        raise NotImplementedError

In [3]:
class Decoder(nn.Module):
    """编码器-解码器架构的基本解码器接口"""
    def __init__(self, **kwargs):
        super(Decoder, self).__init__(**kwargs)

    def init_state(self, enc_outputs, *args):
        raise NotImplementedError

    def forward(self, X, state):
        raise NotImplementedError

In [5]:
class EncoderDecoder(nn.Module):
    """编码器-解码器架构的基类"""
    def __init__(self,encoder,decoder,**kwargs):
        super(EncoderDecoder,self).__init__(**kwargs)
        self.decoder=decoder
        self.encoder=encoder
        
    def forward(self,enc_X,dec_X,*args):
        enc_outputs=self.encoder(enc_X,*args)
        dec_state=self.decoder.init_state(enc_outputs,*args)
        return self.decoder(dec_X,dec_state)

从技术上讲，编码器将长度可变的出入序列转换成形状固定的上下文变量c，并且将输入序列的信息在该上下文变量中进行编码。
可以使用循环神经网络来设置编码器。

In [8]:
class Seq2SeqEncoder(Encoder):
    """用于序列到序列的循环神经网络"""
    def __init__(self,vocab_size,embed_size,num_hiddens,num_layers,dropout=0,**kwargs):
        super(Seq2SeqEncoder, self).__init__(**kwargs)
        self.embedding=nn.Embedding(vocab_size,embed_size)# 嵌入层
        self.gru=nn.GRU(embed_size,num_hiddens,num_layers,dropout=dropout)# GRU层
        
    def forward(self,X,*args):
        X=self.embedding(X)#(batch_size,num_steps,embed_size)
        # 在循环神经网络模型中，第一个轴对应于时间步
        X = X.permute(1, 0, 2)
        # 如果未提及状态，则默认为0
        output, state = self.gru(X)
        # output的形状:(num_steps,batch_size,num_hiddens)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

In [10]:
encoder = Seq2SeqEncoder(vocab_size=10, embed_size=8, num_hiddens=16, num_layers=2)
encoder.eval()
X = torch.zeros((4, 7), dtype=torch.long)
output, state = encoder(X)
output.shape, state.shape

(torch.Size([7, 4, 16]), torch.Size([2, 4, 16]))

In [12]:
class Seq2SeqDecoder(d2l.Decoder):
    """用于序列到序列学习的循环神经网络解码器"""
    def __init__(self, vocab_size, embed_size, num_hiddens, num_layers, dropout=0, **kwargs):
        super(Seq2SeqDecoder, self).__init__(**kwargs)
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru = nn.GRU(embed_size + num_hiddens, num_hiddens, num_layers,
                          dropout=dropout)
        self.dense = nn.Linear(num_hiddens, vocab_size)

    def init_state(self, enc_outputs, *args):
        return enc_outputs[1]

    def forward(self, X, state):
        # 输出'X'的形状：(batch_size,num_steps,embed_size)
        X = self.embedding(X).permute(1, 0, 2)
        # 广播context，使其具有与X相同的num_steps
        context = state[-1].repeat(X.shape[0], 1, 1)
        X_and_context = torch.cat((X, context), 2)
        output, state = self.gru(X_and_context, state)
        output = self.dense(output).permute(1, 0, 2)
        # output的形状:(batch_size,num_steps,vocab_size)
        # state的形状:(num_layers,batch_size,num_hiddens)
        return output, state

```python
context = state[-1].repeat(X.shape[0], 1, 1)
X_and_context = torch.cat((X, context), 2)
```
在Seq2Seq解码器中，context是编码器的最终隐藏状态。它用于将当前时间步的解码器输入与先前的上下文信息结合起来，以便更好地预测下一个时步的输出。具体地说，上下文通过广播（repeat）和连接（concatenation）操作与当前时间步的解码器输入一起馈送到GRU神经元中进行处理。这允许模型同时考虑先前的上下文信息和当前的输入信息，从而产生更准确的输出预测。

先前的上下文信息指的是之前时间步的编码器最终隐藏状态（也称为编码器状态或编码器输出）。在Seq2Seq模型中，编码器将输入序列转换成一个固定长度的向量作为上下文信息，然后解码器使用这个向量作为先前的上下文信息来帮助预测当前时间步的输出。因此，解码器的输入不仅包括当前时间步的输入，还包括先前的上下文信息（通常通过广播和连接操作得到）。

In [13]:
decoder = Seq2SeqDecoder(vocab_size=10, embed_size=8, num_hiddens=16,
                         num_layers=2)
decoder.eval()
state = decoder.init_state(encoder(X))
output, state = decoder(X, state)
output.shape, state.shape

(torch.Size([4, 7, 10]), torch.Size([2, 4, 16]))